### Step 1: Edge detection with canny
Lý do: Ta cần phát hiện cạnh để tách biệt các vật thể + loại bỏ nhiễu, giúp cho việc nhận diện dễ dàng hơn

In [20]:
from imutils.perspective import four_point_transform
from imutils import contours
import numpy as np
import imutils
import cv2

orig = cv2.imread("Image/paper - Copy.jfif")
image = orig.copy()
image = imutils.resize(image, height=1500)

In [21]:
# gray it
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# blur it
gray_blur = cv2.GaussianBlur(gray, (5, 5), 0)
gray_blur_little = cv2.GaussianBlur(gray, (3, 3), 0)

# canny edge detection
# Có Sobel và Prewitt nữa nhưng mà chúng nó nhạy cảm hơn với nhiễu + xử lý tương phản thấp ko tốt
# Canny threshold: nếu pixel < low thì bị loại bỏ, > high chắc chắn được giữ, nếu nằm giữa thì xem xét nó có liên kết với một strong edge không
edged = cv2.Canny(gray_blur, 20, 50)

cv2.imshow("canny", imutils.resize(edged, height=700))
cv2.waitKey(0)
cv2.destroyAllWindows()

### Step 2: Apply a perspective transform to extract the top-down, birds-eye-view of the exam.

#### 2.1: Find contour of exam code

In [22]:
'''
contours_outer_edge = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours_outer_edge = imutils.grab_contours(contours_outer_edge)
sheetCnts = None
count = 0
if len(contours_outer_edge) > 0:
    contours_outer_edge = sorted(contours_outer_edge, key=cv2.contourArea, reverse=True)
    for c in contours_outer_edge:
        # calc the perimeter
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.05*peri, True)
        if len(approx) == 4:
            count += 1
            sheetCnts = approx
            cv2.drawContours(image, [sheetCnts], -1, (0, 255, 0), 2)
            cv2.imshow("point", imutils.resize(image, height=700))
            cv2.waitKey(0)
            cv2.destroyAllWindows()
            break
'''

'\ncontours_outer_edge = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)\ncontours_outer_edge = imutils.grab_contours(contours_outer_edge)\nsheetCnts = None\ncount = 0\nif len(contours_outer_edge) > 0:\n    contours_outer_edge = sorted(contours_outer_edge, key=cv2.contourArea, reverse=True)\n    for c in contours_outer_edge:\n        # calc the perimeter\n        peri = cv2.arcLength(c, True)\n        approx = cv2.approxPolyDP(c, 0.05*peri, True)\n        if len(approx) == 4:\n            count += 1\n            sheetCnts = approx\n            cv2.drawContours(image, [sheetCnts], -1, (0, 255, 0), 2)\n            cv2.imshow("point", imutils.resize(image, height=700))\n            cv2.waitKey(0)\n            cv2.destroyAllWindows()\n            break\n'

In [23]:
contours_outer_edge = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours_outer_edge = imutils.grab_contours(contours_outer_edge)
sheetCnts = []
count = 0

if len(contours_outer_edge) > 2:  # Thay đổi này để đảm bảo rằng có ít nhất 2 đường viền
    contours_outer_edge = sorted(contours_outer_edge, key=cv2.contourArea, reverse=True)
    for c in contours_outer_edge:
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.05 * peri, True)
        if len(approx) == 4:
            count += 1
            sheetCnts.append(approx)
            cv2.drawContours(image, [approx], -1, (0, 255, 0), 2)
            if count == 3:  # Thoát vòng lặp sau khi tìm thấy đường viền lớn thứ hai
                break

cv2.imshow("point", imutils.resize(image, height=700))
cv2.waitKey(0)
cv2.destroyAllWindows()


#### 2.2 Apply four_point_transform to display bird's eyes view (top-down)

In [24]:
exam_code = four_point_transform(image, sheetCnts[2].reshape(4, 2))

warped_code = four_point_transform(gray_blur_little, sheetCnts[2].reshape(4,2))

cv2.imshow("bird eye", imutils.resize(warped_code, height=700))
cv2.waitKey(0)
cv2.destroyAllWindows()

#### 2.3 Using highpass (Laplacian) to sharpen image including contour and change image into binary

In [25]:
# Define the Laplacian kernel for sharpening
kernel = np.array([
    [0, 1, 0],
    [1, -4, 1],
    [0, 1, 0]
])

# Apply the Laplacian filter for sharpening
sharpend = cv2.filter2D(warped_code, -1, kernel)
warped_code = warped_code + sharpend

thresh_exam_code = cv2.adaptiveThreshold(
    warped_code, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)
cv2.imshow("thresh", imutils.resize(thresh_exam_code, height=700))
cv2.waitKey(0)
cv2.destroyAllWindows()

In [26]:
# vẽ viền cho tất cả để xác nhận lại viền có đúng không
cnts = cv2.findContours(thresh_exam_code.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
print(len(cnts))

cv2.drawContours(warped_code, cnts, -1, (0, 255, 0), 1)
cv2.imshow('thresh_exam_code', imutils.resize(warped_code, height=700))
cv2.waitKey(0)
cv2.destroyAllWindows()

127


#### 2.4 find contours of exam code and get it

In [27]:
# Tô các ô đề thi 
exam_codes = []
print(len(cnts))
for c in cnts:
    (x, y, w, h) = cv2.boundingRect(c)
    ar = w / float(h)
    if (y>=20) and (w >= 15 and h >= 15) and (w <= 50 and h <= 50) and ar >= 0.7 and ar <= 1.3:
        exam_codes.append(c)
print(len(exam_codes))


cv2.drawContours(exam_code, exam_codes, -1, (255, 0, 0), 1)
cv2.imshow('exam code', imutils.resize(exam_code, height=700))
cv2.waitKey(0)
cv2.destroyAllWindows()

127
30


In [28]:
# Get exam code via painted bubble
examcodeCnts = contours.sort_contours(exam_codes, method="top-to-bottom")[0]

examcodeCnts_ordered = []
for i in np.arange(0, len(examcodeCnts), 3):
        ordered_exam = contours.sort_contours(examcodeCnts[i:i+3])[0]
        for q in ordered_exam:
             examcodeCnts_ordered.append(q)

print(len(examcodeCnts_ordered))

30


In [29]:
code = [0, 0, 0]  # Sử dụng danh sách thay vì chuỗi để lưu trữ các giá trị mã đề
ordered_code = []
# Sắp xếp các contours cho chỉ số hàng hiện tại từ trái sang phải, sau đó khởi tạo chỉ số của ô mã đề đã được tô
# cnts = contours.sort_contours(examcodeCnts_ordered[i:i + 3])[0]

bubbled = [-1, -1]
# Lặp qua các contours của từng ô mã đề
for (q, i) in enumerate(np.arange(0, len(examcodeCnts_ordered))):
    mask = np.zeros(thresh_exam_code.shape, dtype="uint8")
    cv2.drawContours(mask, examcodeCnts_ordered[i], -1, 255, -1)

    kernel = np.ones((10, 10), np.uint8)
    mask = cv2.dilate(mask, kernel, iterations=1)

    # Áp dụng mask vào ảnh threshold, sau đó đếm số lượng pixel khác không trong vùng bubble
    mask = cv2.bitwise_and(thresh_exam_code, thresh_exam_code, mask=mask)

    total = cv2.countNonZero(mask)
    ordered_code.append(total)

sorted_indexes = sorted(range(len(ordered_code)), key=lambda i: ordered_code[i], reverse=True)[:3]
for q in sorted_indexes:
        # Cập nhật giá trị mã đề tương ứng
    if q in [0, 3, 6, 9, 12, 15, 18, 21, 24, 27]:
        code[0] += (int(q / 3) + 1) % 10
    elif q in [1, 4, 7, 10, 13, 16, 19, 22, 25, 28]:
        code[1] += (int(q / 3) + 1) % 10
    elif q in [2, 5, 8, 11, 14, 17, 20, 23, 26, 29]:
        code[2] += (int(q / 3) + 1) % 10
# Chuyển danh sách code thành chuỗi và in ra
code = int(''.join(str(x) for x in code))
print(code)


242


#### 2.5: Find contour of answer sheet

In [589]:
'''
paper = four_point_transform(image, sheetCnts.reshape(4, 2))
warped = four_point_transform(gray_blur, sheetCnts.reshape(4,2))
cv2.imshow("bird", imutils.resize(paper, height=700))
cv2.waitKey(0)
cv2.destroyAllWindows()
'''

'\npaper = four_point_transform(image, sheetCnts.reshape(4, 2))\nwarped = four_point_transform(gray_blur, sheetCnts.reshape(4,2))\ncv2.imshow("bird", imutils.resize(paper, height=700))\ncv2.waitKey(0)\ncv2.destroyAllWindows()\n'

In [590]:
paper = four_point_transform(image, sheetCnts[0].reshape(4, 2))
warped = four_point_transform(gray_blur, sheetCnts[0].reshape(4,2))
cv2.imshow("bird", imutils.resize(paper, height=700))
cv2.waitKey(0)
cv2.destroyAllWindows()

### Step 3: Extract the set of bubbles (i.e., the possible answer choices) from the perspective transformed exam.

In [591]:
thresh = cv2.adaptiveThreshold(
    warped, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)
cv2.imshow("thresh", imutils.resize(thresh, height=700))
cv2.waitKey(0)
cv2.destroyAllWindows()

In [592]:
# vẽ viền cho tất cả để xác nhận lại viền có đúng không
cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
print(len(cnts))

cv2.drawContours(warped, cnts, -1, (0, 255, 0), 1)
cv2.imshow('thres', imutils.resize(warped, height=700))
cv2.waitKey(0)
cv2.destroyAllWindows()

654


In [593]:
# Tô các đáp án (mỗi câu 4 đáp án)
questions = []
print(len(cnts))
for c in cnts:
    (x, y, w, h) = cv2.boundingRect(c)
    ar = w / float(h)
    if (w >= 15 and h >= 15) and (w <= 50 and h <= 50) and ar >= 0.7 and ar <= 1.3:
        questions.append(c)
print(len(questions))


cv2.drawContours(paper, questions, -1, (255, 0, 0), 1)
cv2.imshow('questions', imutils.resize(paper, height=700))
cv2.waitKey(0)
cv2.destroyAllWindows()

654
240


### Step 4: Get answer list, get question contours and arrange them to mark

#### 4.1 Get question index and answer key respetively

In [594]:
import os
from tkinter import filedialog
from openpyxl import load_workbook
# Create an empty dictionary to store answers
answers_dict = {}
def get_answer_indices(excel_path):
    # Load the workbook and select the active worksheet
    workbook = load_workbook(excel_path)
    sheet = workbook.active
    
    # Create a dictionary to map letters to indices
    answer_map = {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4}
    
    # Create a list to store the answer indices
    answer_indices = []
    
    # Assuming the first column has question numbers and the second has answers
    for row in sheet.iter_rows(min_row=1):  # Skip the header row
        answer_letter = row[1].value
        # Convert the letter to an index using the answer_map
        answer_index = answer_map.get(answer_letter, -1)  # Default to -1 if not found
        answer_indices.append(answer_index)
    
    return answer_indices

def read_answers_in_directory(directory_path):
    # Get a list of all files in the directory
    for filename in os.listdir(directory_path):
        if filename.endswith(".xlsx"):
            file_path = os.path.join(directory_path, filename)
            # Extract the exam code from the filename
            ma_de = filename.split(".xlsx")[0]
            answer_indices = get_answer_indices(file_path)
            answers_dict[ma_de] = answer_indices
    
    return answers_dict

def choose_directory(): 
    # Prompt the user to select a directory
    directory_path = filedialog.askdirectory()
    return directory_path

# Example usage
directory_path = choose_directory()
if directory_path:
    answers_dict = read_answers_in_directory(directory_path)
    print("Answers for each exam code':")
    for ma_de, answer_indices in answers_dict.items():
        print(f"{ma_de}: {answer_indices}")
else:
    print("No directory selected.")


Answers for each exam code':
149: [3, 1, 2, 0, 0, 3, 2, 0, 3, 1, 1, 0, 0, 2, 3, 1, 0, 1, 1, 2, 2, 3, 0, 1, 2, 2, 3, 0, 1, 3, 0, 2, 1, 3, 0, 2, 1, 3, 2, 1, 0, 2, 2, 3, 0, 3, 2, 1, 1, 0, 0, 3, 2, 1, 3, 0, 3, 1, 2, 0]


In [595]:
# Answer Key
ques = [i for i in range(60)]

for ma_de, answer_indices in answers_dict.items():
    if code == int(ma_de):
        opts = answer_indices
print(opts)
print(len(opts))

ANSWER_KEY = dict(zip(ques, opts))

[3, 1, 2, 0, 0, 3, 2, 0, 3, 1, 1, 0, 0, 2, 3, 1, 0, 1, 1, 2, 2, 3, 0, 1, 2, 2, 3, 0, 1, 3, 0, 2, 1, 3, 0, 2, 1, 3, 2, 1, 0, 2, 2, 3, 0, 3, 2, 1, 1, 0, 0, 3, 2, 1, 3, 0, 3, 1, 2, 0]
60


#### 4.2 Get question contours and arrange them in order: top-down questions, right-to-left answers

In [596]:
# Get question contours and order
questionCnts = contours.sort_contours(questions, method="top-to-bottom")[0]

questionCnts_ordered = []


for i in np.arange(0, len(questionCnts), 16):
        ordered = contours.sort_contours(questionCnts[i:i+16])[0]
        for q in ordered:
             questionCnts_ordered.append(q)

bubble_thresh = 185
correct = 0
# Lặp qua các contours của từng câu hỏi
for (q, i) in enumerate(np.arange(0, len(questionCnts_ordered), 4)):
    # Sắp xếp các contours cho câu hỏi hiện tại từ trái sang phải, sau đó khởi tạo chỉ số của câu trả lời đã được chọn
    cnts = contours.sort_contours(questionCnts_ordered[i:i + 4])[0]
    bubbled = [-1, -1]
    bubble_count = 0

    # Lặp qua các contours đã được sắp xếp
    for (j, c) in enumerate(cnts):
        # Tạo mask để chỉ hiển thị "bubbles" cho câu hỏi hiện tại
        mask = np.zeros(thresh.shape, dtype="uint8")
        cv2.drawContours(mask, [c], -1, 255, -1)

        # Áp dụng mask vào ảnh threshold, sau đó đếm số lượng pixel khác không trong vùng bubble
        mask = cv2.bitwise_and(thresh, thresh, mask=mask)
        total = cv2.countNonZero(mask)

        # if total > current bubbled then
        # bubbled = total
        if bubbled[0] == -1 or bubbled[0] < total:
            bubbled = (total, j)
            if bubbled[0] > bubble_thresh:
                bubble_count += 1
        else:
            # update bubble thresh
            # if total <= bubbled[0]
            # this means that bubbled is not filled
            # therefore we are updating bubble thresh to adjust to the pic
            bubble_thresh = max(bubble_thresh, total+5)

    # Khởi tạo màu của contour và chỉ số của đáp án
    color = (0, 0, 255)
    row = q // 4
    col = q % 4
    q =col * 15 + row 
    k = ANSWER_KEY[q]

    # Kiểm tra xem câu trả lời đã được chọn có đúng không
    if k == bubbled[1] and bubbled[0] > bubble_thresh:
        color = (0, 255, 0)
        correct += 1
    # Vẽ viền của câu trả lời đúng trên bài kiểm tra
    cv2.drawContours(paper, [cnts[k]], -1, color, 3)
print(correct)
cv2.imshow('show answer', imutils.resize(paper, height=700))
cv2.waitKey(0)
cv2.destroyAllWindows()

3


### Step 5: grading 

In [597]:
# số câu tối đa
max_marks = 60
# số điểm cho mỗi câu đúng
positive_marking = 1
# số điểm cho mỗi câu sai
# negative_marking = 0

# grab the test taker
# Tính toán điểm số 
score = (correct*positive_marking) / max_marks * 100

# Thông tin về số câu trả lời đúng và điểm số được lưu vào một dictionary info
info = {
    "correct": correct,
    #"wrong": wrong,
    "score": score
}
print(info)

{'correct': 3, 'score': 5.0}


In [598]:
# Hiển thị số lượng câu trả lời đúng, sai và điểm số lên trang giấy
cv2.putText(paper, "Correct: {}".format(correct), (10, 30),
            cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
#cv2.putText(paper, "Wrong: {}".format(wrong), (10, 60),
#            cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
cv2.putText(paper, "Score: {:.1f}%".format(score), (180, 30),
            cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

cv2.imshow('show score', imutils.resize(paper, height=700))
cv2.waitKey(0)
cv2.destroyAllWindows()